In [1]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

import random
from scipy.spatial.distance import cdist
from sklearn.preprocessing import Imputer
from collections import defaultdict
import itertools
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('allPatients copy.csv',low_memory=False)

In [3]:
X = df.drop('Classes',axis=1)
features = X.columns
X.replace(to_replace='?', value = 'NaN', inplace=True)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X=imp.fit_transform(X)
Y = df['Classes']
X = pd.DataFrame(X,columns=features)

In [13]:
clf = RandomForestClassifier(n_estimators=300, criterion = 'entropy', max_features = 100)
clf.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=100, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
freqFeatures = np.zeros(shape=(8560), dtype=int)
for tree in clf:
    for i in tree.tree_.feature:
        if i !=-2:
            freqFeatures[i] = freqFeatures[i]+1

In [15]:
mostFreqFeatures = np.argsort(freqFeatures)
top200 = mostFreqFeatures[8360:8560]
pd.DataFrame(top200).to_csv('/Users/Daniel/Documents/Data Mining 514/HW2/top200Features.csv')
featureSelectedX = X.ix[:,top200]

In [16]:
def getClusterGroups(labels,data,k):
    d = dict(); 
    for i in range(0,k):
        d["cluster"+str(i)] =  data.ix[np.nonzero(labels==i)]
    return d

def getDistances(clusters,distanceMeasure):
    distDict = dict()
    for cluster1 in itertools.combinations(clusters, 2):
        dist = cdist(clusters[cluster1[0]],clusters[cluster1[1]], distanceMeasure)
        n,m = dist.shape
        distDict[cluster1[0]+" " +cluster1[1] + " Single Link"] = np.nanmin(dist)
        distDict[cluster1[0]+" " +cluster1[1] + " complete Link"] = np.nanmax(dist)
        dist = np.nan_to_num(dist)
        distDict[cluster1[0]+" " + cluster1[1] + " average"] = sum(sum(dist))/(n*m)
        distDict[cluster1[0] + " "+ cluster1[1] + " centriod"] = np.amax(cdist(pd.DataFrame(clusters[cluster1[0]].mean()).transpose(),pd.DataFrame(clusters[cluster1[1]].mean()).transpose(),distanceMeasure))
    return distDict

In [17]:
estimatorsEuclidean = {'k_means_euclidean_2': KMeans(n_clusters=2).fit(featureSelectedX),
              'k_means_euclidean_3': KMeans(n_clusters=3).fit(featureSelectedX),
              'k_means_euclidean_4': KMeans(n_clusters=4).fit(featureSelectedX)}

def new_euclidean_distances(X, Y=None, Y_norm_squared=None, squared=False): 
    return cosine_similarity(X,Y)

# monkey patch (ensure cosine dist function is used)
KMeans.euclidean_distances = new_euclidean_distances 
estimatorsDot = {'k_means_dot_2': KMeans(n_clusters=2).fit(featureSelectedX),
              'k_means_dot_3': KMeans(n_clusters=3).fit(featureSelectedX),
              'k_means_dot_4': KMeans(n_clusters=4).fit(featureSelectedX)}

In [18]:
for cluster in estimatorsEuclidean:
    k = len(np.unique(estimatorsEuclidean[cluster].labels_))
    groups = getClusterGroups(estimatorsEuclidean[cluster].labels_,featureSelectedX,len(np.unique(estimatorsEuclidean[cluster].labels_)))
    dist = getDistances(groups,'euclidean')
    pd.DataFrame([dist]).transpose().to_csv('/results/ResultsEuclideanK({0}).csv'.format(k))
for cluster in estimatorsDot:
    k = len(np.unique(estimatorsDot[cluster].labels_))
    groups = getClusterGroups(estimatorsDot[cluster].labels_,featureSelectedX,k)
    dist = getDistances(groups,'cosine')
    pd.DataFrame([dist]).transpose().to_csv('/results/ResultsDotK({0}).csv'.format(k))
